In [69]:
import os
import json
import pandas as pd

folder_path = '/home/jupyter-23521027/refresh-bert/data/liputan6_data/canonical/train' 

file_list = os.listdir(folder_path)

In [70]:
json_data_list = []

file_list = os.listdir(folder_path)

for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)  
    
    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            json_data = json.load(file)
            
            json_data_list.append(json_data)

df = pd.DataFrame(json_data_list)

In [71]:
df

,id,url,clean_article,clean_summary,extractive_summary
0,48951,https://www.liputan6.com/news/read/48951/polis...,"[[Liputan6, ., com, ,, Lamongan, :, Tim, Inves...","[[Senjata, api, dan, ratusan, butir, peluru, m...","[1, 2]"
1,124973,https://www.liputan6.com/news/read/124973/gent...,"[[Liputan6, ., com, ,, Depok, :, Abdul, Madjid...","[[Genta, buatan, Abdul, Madjid, bisa, melahirk...","[0, 1]"
2,298677,https://www.liputan6.com/news/read/298677/wasp...,"[[Liputan6, ., com, ,, Bekasi, :, Aparat, Kepo...","[[Berhati-hatilah, Anda, bila, bepergian, ke, ...","[0, 5]"
3,40569,https://www.liputan6.com/news/read/40569/imbau...,"[[Liputan6, ., com, ,, Denpasar, :, Imbauan, M...","[[Peringatan, untuk, tidak, pergi, ke, Indones...","[5, 8]"
4,242869,https://www.liputan6.com/news/read/242869/rapa...,"[[Liputan6, ., com, ,, Jakarta, :, Gempa, kuat...","[[Rapat, pleno, KPU, terhenti, ketika, gempa, ...","[0, 4]"
...,...,...,...,...,...
193878,164143,https://www.liputan6.com/news/read/164143/tahu...,"[[Liputan6, ., com, ,, Jakarta, :, Warga, yang...","[[Dirjen, Pajak, membuat, aturan, setiap, oran...","[2, 4]"
193879,150247,https://www.liputan6.com/news/read/150247/mass...,"[[Liputan6, ., com, ,, Bogor, :, Ribuan, umat,...","[[Ribuan, umat, Islam, menyerbu, vila, milik, ...","[0, 1]"
193880,204203,https://www.liputan6.com/news/read/204203/bate...,"[[Chelsea, melayangkan, keberatannya, kepada, ...","[[Ketua, Leeds, United, ,, Ken, Bates, berhasi...","[0, 13]"
193881,159188,https://www.liputan6.com/news/read/159188/peny...,"[[Liputan6, ., com, ,, Denpasar, :, Kebakaran,...","[[Kebakaran, toko, sepatu, di, Denpasar, ,, Ba...","[0, 3]"


In [72]:
df['sentences'] = df['clean_article'].apply(lambda x: [' '.join(sentence) for sentence in x])

In [73]:
df

,id,url,clean_article,clean_summary,extractive_summary,sentences
0,48951,https://www.liputan6.com/news/read/48951/polis...,"[[Liputan6, ., com, ,, Lamongan, :, Tim, Inves...","[[Senjata, api, dan, ratusan, butir, peluru, m...","[1, 2]","[Liputan6 . com , Lamongan : Tim Investigasi B..."
1,124973,https://www.liputan6.com/news/read/124973/gent...,"[[Liputan6, ., com, ,, Depok, :, Abdul, Madjid...","[[Genta, buatan, Abdul, Madjid, bisa, melahirk...","[0, 1]","[Liputan6 . com , Depok : Abdul Madjid punya k..."
2,298677,https://www.liputan6.com/news/read/298677/wasp...,"[[Liputan6, ., com, ,, Bekasi, :, Aparat, Kepo...","[[Berhati-hatilah, Anda, bila, bepergian, ke, ...","[0, 5]","[Liputan6 . com , Bekasi : Aparat Kepolisian R..."
3,40569,https://www.liputan6.com/news/read/40569/imbau...,"[[Liputan6, ., com, ,, Denpasar, :, Imbauan, M...","[[Peringatan, untuk, tidak, pergi, ke, Indones...","[5, 8]","[Liputan6 . com , Denpasar : Imbauan Menteri L..."
4,242869,https://www.liputan6.com/news/read/242869/rapa...,"[[Liputan6, ., com, ,, Jakarta, :, Gempa, kuat...","[[Rapat, pleno, KPU, terhenti, ketika, gempa, ...","[0, 4]","[Liputan6 . com , Jakarta : Gempa kuat yang me..."
...,...,...,...,...,...,...
193878,164143,https://www.liputan6.com/news/read/164143/tahu...,"[[Liputan6, ., com, ,, Jakarta, :, Warga, yang...","[[Dirjen, Pajak, membuat, aturan, setiap, oran...","[2, 4]","[Liputan6 . com , Jakarta : Warga yang belum m..."
193879,150247,https://www.liputan6.com/news/read/150247/mass...,"[[Liputan6, ., com, ,, Bogor, :, Ribuan, umat,...","[[Ribuan, umat, Islam, menyerbu, vila, milik, ...","[0, 1]","[Liputan6 . com , Bogor : Ribuan umat Islam me..."
193880,204203,https://www.liputan6.com/news/read/204203/bate...,"[[Chelsea, melayangkan, keberatannya, kepada, ...","[[Ketua, Leeds, United, ,, Ken, Bates, berhasi...","[0, 13]",[Chelsea melayangkan keberatannya kepada Asosi...
193881,159188,https://www.liputan6.com/news/read/159188/peny...,"[[Liputan6, ., com, ,, Denpasar, :, Kebakaran,...","[[Kebakaran, toko, sepatu, di, Denpasar, ,, Ba...","[0, 3]","[Liputan6 . com , Denpasar : Kebakaran toko se..."


In [74]:
df.drop('clean_article', axis=1, inplace=True)

In [75]:
df

,id,url,clean_summary,extractive_summary,sentences
0,48951,https://www.liputan6.com/news/read/48951/polis...,"[[Senjata, api, dan, ratusan, butir, peluru, m...","[1, 2]","[Liputan6 . com , Lamongan : Tim Investigasi B..."
1,124973,https://www.liputan6.com/news/read/124973/gent...,"[[Genta, buatan, Abdul, Madjid, bisa, melahirk...","[0, 1]","[Liputan6 . com , Depok : Abdul Madjid punya k..."
2,298677,https://www.liputan6.com/news/read/298677/wasp...,"[[Berhati-hatilah, Anda, bila, bepergian, ke, ...","[0, 5]","[Liputan6 . com , Bekasi : Aparat Kepolisian R..."
3,40569,https://www.liputan6.com/news/read/40569/imbau...,"[[Peringatan, untuk, tidak, pergi, ke, Indones...","[5, 8]","[Liputan6 . com , Denpasar : Imbauan Menteri L..."
4,242869,https://www.liputan6.com/news/read/242869/rapa...,"[[Rapat, pleno, KPU, terhenti, ketika, gempa, ...","[0, 4]","[Liputan6 . com , Jakarta : Gempa kuat yang me..."
...,...,...,...,...,...
193878,164143,https://www.liputan6.com/news/read/164143/tahu...,"[[Dirjen, Pajak, membuat, aturan, setiap, oran...","[2, 4]","[Liputan6 . com , Jakarta : Warga yang belum m..."
193879,150247,https://www.liputan6.com/news/read/150247/mass...,"[[Ribuan, umat, Islam, menyerbu, vila, milik, ...","[0, 1]","[Liputan6 . com , Bogor : Ribuan umat Islam me..."
193880,204203,https://www.liputan6.com/news/read/204203/bate...,"[[Ketua, Leeds, United, ,, Ken, Bates, berhasi...","[0, 13]",[Chelsea melayangkan keberatannya kepada Asosi...
193881,159188,https://www.liputan6.com/news/read/159188/peny...,"[[Kebakaran, toko, sepatu, di, Denpasar, ,, Ba...","[0, 3]","[Liputan6 . com , Denpasar : Kebakaran toko se..."


In [76]:
df['summary_sentences'] = df['clean_summary'].apply(lambda x: [' '.join(sentence) for sentence in x])

In [77]:
df

,id,url,clean_summary,extractive_summary,sentences,summary_sentences
0,48951,https://www.liputan6.com/news/read/48951/polis...,"[[Senjata, api, dan, ratusan, butir, peluru, m...","[1, 2]","[Liputan6 . com , Lamongan : Tim Investigasi B...",[Senjata api dan ratusan butir peluru milik te...
1,124973,https://www.liputan6.com/news/read/124973/gent...,"[[Genta, buatan, Abdul, Madjid, bisa, melahirk...","[0, 1]","[Liputan6 . com , Depok : Abdul Madjid punya k...",[Genta buatan Abdul Madjid bisa melahirkan alu...
2,298677,https://www.liputan6.com/news/read/298677/wasp...,"[[Berhati-hatilah, Anda, bila, bepergian, ke, ...","[0, 5]","[Liputan6 . com , Bekasi : Aparat Kepolisian R...",[Berhati-hatilah Anda bila bepergian ke luar r...
3,40569,https://www.liputan6.com/news/read/40569/imbau...,"[[Peringatan, untuk, tidak, pergi, ke, Indones...","[5, 8]","[Liputan6 . com , Denpasar : Imbauan Menteri L...",[Peringatan untuk tidak pergi ke Indonesia tak...
4,242869,https://www.liputan6.com/news/read/242869/rapa...,"[[Rapat, pleno, KPU, terhenti, ketika, gempa, ...","[0, 4]","[Liputan6 . com , Jakarta : Gempa kuat yang me...",[Rapat pleno KPU terhenti ketika gempa kuat me...
...,...,...,...,...,...,...
193878,164143,https://www.liputan6.com/news/read/164143/tahu...,"[[Dirjen, Pajak, membuat, aturan, setiap, oran...","[2, 4]","[Liputan6 . com , Jakarta : Warga yang belum m...",[Dirjen Pajak membuat aturan setiap orang berp...
193879,150247,https://www.liputan6.com/news/read/150247/mass...,"[[Ribuan, umat, Islam, menyerbu, vila, milik, ...","[0, 1]","[Liputan6 . com , Bogor : Ribuan umat Islam me...",[Ribuan umat Islam menyerbu vila milik jamaah ...
193880,204203,https://www.liputan6.com/news/read/204203/bate...,"[[Ketua, Leeds, United, ,, Ken, Bates, berhasi...","[0, 13]",[Chelsea melayangkan keberatannya kepada Asosi...,"[Ketua Leeds United , Ken Bates berhasil lolos..."
193881,159188,https://www.liputan6.com/news/read/159188/peny...,"[[Kebakaran, toko, sepatu, di, Denpasar, ,, Ba...","[0, 3]","[Liputan6 . com , Denpasar : Kebakaran toko se...","[Kebakaran toko sepatu di Denpasar , Bali , ya..."


In [78]:
df['summary_string'] = df['summary_sentences'].apply(lambda x: ' '.join(x))

In [79]:
df.drop('clean_summary', axis=1, inplace=True)
df.drop('summary_sentences', axis=1, inplace=True)

In [80]:
df

,id,url,extractive_summary,sentences,summary_string
0,48951,https://www.liputan6.com/news/read/48951/polis...,"[1, 2]","[Liputan6 . com , Lamongan : Tim Investigasi B...",Senjata api dan ratusan butir peluru milik ter...
1,124973,https://www.liputan6.com/news/read/124973/gent...,"[0, 1]","[Liputan6 . com , Depok : Abdul Madjid punya k...",Genta buatan Abdul Madjid bisa melahirkan alun...
2,298677,https://www.liputan6.com/news/read/298677/wasp...,"[0, 5]","[Liputan6 . com , Bekasi : Aparat Kepolisian R...",Berhati-hatilah Anda bila bepergian ke luar ru...
3,40569,https://www.liputan6.com/news/read/40569/imbau...,"[5, 8]","[Liputan6 . com , Denpasar : Imbauan Menteri L...",Peringatan untuk tidak pergi ke Indonesia tak ...
4,242869,https://www.liputan6.com/news/read/242869/rapa...,"[0, 4]","[Liputan6 . com , Jakarta : Gempa kuat yang me...",Rapat pleno KPU terhenti ketika gempa kuat men...
...,...,...,...,...,...
193878,164143,https://www.liputan6.com/news/read/164143/tahu...,"[2, 4]","[Liputan6 . com , Jakarta : Warga yang belum m...",Dirjen Pajak membuat aturan setiap orang berpe...
193879,150247,https://www.liputan6.com/news/read/150247/mass...,"[0, 1]","[Liputan6 . com , Bogor : Ribuan umat Islam me...",Ribuan umat Islam menyerbu vila milik jamaah A...
193880,204203,https://www.liputan6.com/news/read/204203/bate...,"[0, 13]",[Chelsea melayangkan keberatannya kepada Asosi...,"Ketua Leeds United , Ken Bates berhasil lolos ..."
193881,159188,https://www.liputan6.com/news/read/159188/peny...,"[0, 3]","[Liputan6 . com , Denpasar : Kebakaran toko se...","Kebakaran toko sepatu di Denpasar , Bali , yan..."


In [81]:
# https://github.com/google-research/google-research/blob/master/rouge/rouge_scorer.py
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(
    'IDX Incubator \xa0  mengadakan sesi diskusi dengan tema “ Technology vs Humanity ” ',
    'IDX Incubator adakan sesi diskusi dengan tema Technology vs Humanity'
)
print(scores)

{'rouge1': Score(precision=0.9, recall=0.9, fmeasure=0.9), 'rouge2': Score(precision=0.7777777777777778, recall=0.7777777777777778, fmeasure=0.7777777777777778), 'rougeL': Score(precision=0.9, recall=0.9, fmeasure=0.9)}


In [84]:
from datetime import datetime

DATASET_SIZE = 5000
all_document_sentences_f1_rouge_scores = []

now = datetime.now()
df = df.head(DATASET_SIZE)

for i, row in df.iterrows():
    document_sentences_f1_rouge_scores = []
    for sentence in row["sentences"]:
        scores = scorer.score(sentence, row["summary_string"])
        rouge1_f1 = scores["rouge1"].fmeasure
        rouge2_f1 = scores["rouge2"].fmeasure
        rougeL_f1 = scores["rougeL"].fmeasure
        rouge_score = (rouge1_f1 + rouge2_f1 + rougeL_f1) / 3.0
        document_sentences_f1_rouge_scores.append(rouge_score)
        
    if (i + 1) % 1000 == 0:
        print(f"processing {i + 1} with time: {datetime.now() - now}")
        now = datetime.now()
        
    all_document_sentences_f1_rouge_scores.append(document_sentences_f1_rouge_scores)

df["document_sentences_f1_rouge_scores"] = all_document_sentences_f1_rouge_scores
df.tail(5)

processing 1000 with time: 0:00:10.894322
processing 2000 with time: 0:00:11.027054
processing 3000 with time: 0:00:10.665143
processing 4000 with time: 0:00:10.408754
processing 5000 with time: 0:00:10.648285


/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,url,extractive_summary,sentences,summary_string,document_sentences_f1_rouge_scores
4995,161517,https://www.liputan6.com/news/read/161517/pemb...,"[0, 4, 3]","[Liputan6 . com , Tangerang : Seorang pembantu...",Mulyati ditemukan tewas mengenaskan di sebuah ...,"[0.3164696611505122, 0.08421052631578947, 0.12..."
4996,252163,https://www.liputan6.com/news/read/252163/sigi...,"[0, 5]","[Liputan6 . com , Tangerang : Sigid Haryo Wibi...","Sigid Haryo Wibisono , terdakwa kasus pembunuh...","[0.7612293144208038, 0.0, 0.0761904761904762, ..."
4997,266451,https://www.liputan6.com/news/read/266451/dpr-...,"[2, 7]","[Liputan6 . com , Jakarta : Presiden Susilo Ba...","Presiden SBY menyatakan , DPR menyetujui adany...","[0.04938271604938272, 0.11370082624244666, 0.2..."
4998,138172,https://www.liputan6.com/news/read/138172/truk...,"[0, 1]","[Liputan6 . com , Sidoarjo : Truk pengangkut p...",Truk pengangkut pasir dan batu untuk penguatan...,"[0.5864197530864197, 0.10526315789473684, 0.07..."
4999,268940,https://www.liputan6.com/news/read/268940/gus-...,"[4, 8]","[Liputan6 . com , Surabaya : Muktamar ke-32 Na...",Wakil Ketua DPP PKB Effendi Choirie berharap p...,"[0.5113960113960114, 0.3146953405017921, 0.257..."


In [85]:
all_top_10_sentences = [] 
for i, row in df.iterrows():
    document_sentences_and_scores = [] 
    for sentences_and_scores in zip(
        row["sentences"], 
        row["document_sentences_f1_rouge_scores"], 
        [x+1 for x in range(len(row["sentences"]))]
    ):
        document_sentences_and_scores.append(sentences_and_scores)

    top_10_sentences = sorted(document_sentences_and_scores, key=lambda x: x[1], reverse=True)[:10]
    all_top_10_sentences.append([(x[0],x[2]) for x in top_10_sentences])

df["top_10_sentences"] = all_top_10_sentences
df.head(1)

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,url,extractive_summary,sentences,summary_string,document_sentences_f1_rouge_scores,top_10_sentences
0,48951,https://www.liputan6.com/news/read/48951/polis...,"[1, 2]","[Liputan6 . com , Lamongan : Tim Investigasi B...",Senjata api dan ratusan butir peluru milik ter...,"[0.2001063264221159, 0.42203548085901027, 0.12...",[(Senjata dan peluru itu ditemukan di dalam pi...


In [87]:
import itertools

df_candidate_summary = pd.DataFrame()

all_candidate_summary_txt = []
all_candidate_summary_ids_txt = []
all_document_length = [] 
for i, row in df.iterrows():
    document_candidate_summaries = []
    document_candidate_summaries_idx = []

    for itemcount in [1,2,3]:
        candidate_summaries = list(itertools.combinations(list(row["top_10_sentences"]), itemcount))
        for candidate_summary in candidate_summaries:
            candidate_summary_txt = " ".join([val[0] for val in candidate_summary])
            candidate_summary_ids = " ".join([str(val[1]) for val in candidate_summary])

            document_candidate_summaries.append(candidate_summary_txt)
            document_candidate_summaries_idx.append(candidate_summary_ids)

    all_candidate_summary_txt.append(document_candidate_summaries)
    all_candidate_summary_ids_txt.append(document_candidate_summaries_idx)
    all_document_length.append(len(row["sentences"]))

df_candidate_summary["candidate_summaries"] = all_candidate_summary_txt
df_candidate_summary["candidate_summary_ids"] = all_candidate_summary_ids_txt
df_candidate_summary["document_length"] = all_document_length
df_candidate_summary["summary"] = df["summary_string"]
df_candidate_summary.head(1)

,candidate_summaries,candidate_summary_ids,document_length,summary
0,[Senjata dan peluru itu ditemukan di dalam pip...,"[2, 9, 1, 5, 6, 4, 3, 7, 8, 2 9, 2 1, 2 5, 2 6...",9,Senjata api dan ratusan butir peluru milik ter...


In [88]:
# input: list of kalimat dalam dokumen dan summary
# output: list of scores 

documents = list(
    zip(
        df_candidate_summary["candidate_summaries"], 
        df_candidate_summary["summary"]
    )
)

def cal_rouge(document):
    candidate_summaries = document[0]
    summary = document[1]
    candidate_summary_scores = []
    for candidate_summary in candidate_summaries:
        scores = scorer.score(candidate_summary, summary)
        rouge1_f1 = scores["rouge1"].fmeasure
        rouge2_f1 = scores["rouge2"].fmeasure
        rougeL_f1 = scores["rougeL"].fmeasure
        rouge_score = (rouge1_f1 + rouge2_f1 + rougeL_f1) / 3.0
        candidate_summary_scores.append(rouge_score)
        
    return candidate_summary_scores


In [89]:
import multiprocess
from multiprocess import Pool
import tqdm

cores = 4
print(f"multiprocessing with {cores} cores")


with Pool(processes=cores) as pool:
    results = list(tqdm.tqdm(pool.imap_unordered(cal_rouge, documents), total=len(documents)))

df_candidate_summary["candidate_summary_scores"] = results
df_candidate_summary.tail(5)

multiprocessing with 4 cores


100%|██████████| 5000/5000 [04:50<00:00, 17.19it/s]


,candidate_summaries,candidate_summary_ids,document_length,summary,candidate_summary_scores
4995,[Diduga pelaku sempat memperkosa sebelum membu...,"[5, 1, 4, 3, 7, 12, 6, 2, 17, 10, 5 1, 5 4, 5 ...",18,Mulyati ditemukan tewas mengenaskan di sebuah ...,"[0.3486291486291486, 0.3164696611505122, 0.192..."
4996,"[Liputan6 . com , Tangerang : Sigid Haryo Wibi...","[1, 4, 3, 9, 6, 8, 5, 7, 10, 2, 1 4, 1 3, 1 9,...",10,"Sigid Haryo Wibisono , terdakwa kasus pembunuh...","[0.7612293144208038, 0.1081081081081081, 0.076..."
4997,[Lembaga ini menyetujui Perppu perbaikan perat...,"[8, 3, 4, 2, 9, 6, 7, 10, 5, 1, 8 3, 8 4, 8 2,...",12,"Presiden SBY menyatakan , DPR menyetujui adany...","[0.5864197530864197, 0.19852941176470587, 0.10..."
4998,"[Liputan6 . com , Sidoarjo : Truk pengangkut p...","[1, 5, 2, 7, 3, 8, 4, 6, 1 5, 1 2, 1 7, 1 3, 1...",8,Truk pengangkut pasir dan batu untuk penguatan...,"[0.42597402597402595, 0.29183135704874835, 0.1..."
4999,"[Liputan6 . com , Surabaya : Muktamar ke-32 Na...","[1, 2, 12, 3, 13, 9, 11, 5, 6, 4, 1 2, 1 12, 1...",14,Wakil Ketua DPP PKB Effendi Choirie berharap p...,"[0.5113960113960114, 0.3146953405017921, 0.312..."


In [90]:
all_top_10_candidates = [] 
for i, row in df_candidate_summary.iterrows():
    document_candidates_and_scores = [] 
    for candidates_and_scores in zip(
        row["candidate_summary_ids"], 
        row["candidate_summary_scores"], 
    ):
        document_candidates_and_scores.append(candidates_and_scores)

    top_10_candidates = sorted(document_candidates_and_scores, key=lambda x: x[1], reverse=True)[:10]
    all_top_10_candidates.append(top_10_candidates)

df_candidate_summary["top_10_candidates"] = all_top_10_candidates
df_candidate_summary.head(2)

,candidate_summaries,candidate_summary_ids,document_length,summary,candidate_summary_scores,top_10_candidates
0,[Senjata dan peluru itu ditemukan di dalam pip...,"[2, 9, 1, 5, 6, 4, 3, 7, 8, 2 9, 2 1, 2 5, 2 6...",9,Senjata api dan ratusan butir peluru milik ter...,"[0.2867383512544803, 0.17205520892418227, 0.16...","[(2 4, 0.31227850740045865), (2 6 4, 0.3107692..."
1,"[Warga Sawangan , Depok , Jawa Barat itu mampu...","[2, 8, 6, 4, 10, 11, 1, 7, 3, 9, 2 8, 2 6, 2 4...",11,Genta buatan Abdul Madjid bisa melahirkan alun...,"[0.33771929824561403, 0.19163059163059162, 0.1...","[(2, 0.33771929824561403), (2 1, 0.33695652173..."


In [93]:
df_candidate_summary["id"] = df["id"]

with open(f"/home/jupyter-23521027/refresh-bert/data/preprocessed-input-directory/liputan6.train.label.multipleoracle", "w") as file:
    for i, row in df_candidate_summary.iterrows():
        file.write(f"liputan6-{row['id']}\n")
        file.write(f"{row['document_length']}\n")
        for candidate_summary_tuple in row["top_10_candidates"]:
            file.write(f"{candidate_summary_tuple[0]} {str(candidate_summary_tuple[1])[:14]}\n")
        file.write("\n")

In [94]:
!cat /home/jupyter-23521027/refresh-bert/data/preprocessed-input-directory/liputan6.train.label.multipleoracle | head -c 10000

liputan6-48951
9
2 4 0.312278507400
2 6 4 0.310769230769
2 6 3 0.302214502822
2 1 7 0.298724954462
2 3 7 0.297948717948
9 5 4 0.297598789941
2 5 0.296018344798
2 1 8 0.291187739463
2 0.286738351254
2 1 0.284615384615

liputan6-124973
11
2 0.337719298245
2 1 0.336956521739
2 6 0.330532212885
2 6 1 0.319555952026
2 4 0.317918362313
2 4 1 0.308974715198
2 11 0.305317324185
2 3 0.303321328531
2 11 1 0.298378003296
2 8 1 0.297304806890

liputan6-298677
10
1 6 0.454190186712
1 6 7 0.396825396825
1 6 2 0.383004926108
1 6 5 0.383004926108
1 0.371929824561
1 6 9 0.370114942528
1 6 3 0.370114942528
1 6 8 0.368817204301
1 6 4 0.367607526881
1 6 10 0.346275252525

liputan6-40569
10
9 0.422035480859
9 10 0.403065134099
9 2 0.364425770308
9 6 0.361579161579
9 4 0.351635007372
9 5 6 0.344708994708
9 1 0.335470085470
9 1 2 0.331482198975
9 1 7 0.331086634884
9 2 7 0.326326326326

liputan6-242869
10
1 0.680555555555
5 0.649444444444
8 0.604890604890
10 9 0.589594053744
10 5 0.513834677769
3 0.495779500